In [ ]:
import pandas as pd
import numpy as np

class ScaledWindowedVariance:
    """
    This implementation is based on the paper:
    "Evaluating scaled windowed variance methods for estimating
    the Hurst coefficient of time series" from Physica A. 1997 July 15; 241(3-4): 606–626.
    """
    def __init__(self, data: pd.Series, 
                 min_window: int = 2, 
                 max_window: int = None, 
                 method: str = 'SD', 
                 exclusions: bool = False, 
                 custom_window_list: list = None):
        """
        ScaledWindowedVariance calculates the scaled windowed variance of a time series.
        It allows for different methods of calculation and can exclude certain values based on the method.

        Parameters:
        data (pd.Series): The time series data to analyze.
        min_window (int): The minimum window size for the rolling calculation.
        max_window (int): The maximum window size for the rolling calculation.
        method (str): The method of detrending to use ('SD', 'linear', 'bridge').
        exclusions (bool): Whether to exclude a part of the scaled window according to the method.
        custom_window_list (list): A custom list of window sizes to use instead of the default range.
        """

        self.data = data
        self.exclusions = exclusions
        self.method = method

        if method not in ['SD', 'LD', 'BD']:
            raise ValueError("Method must be one of 'SD', 'LD' or 'BD'.")
        
        N = len(data)
        if N < 2:
            raise ValueError("Data must contain at least two points.")
        
        self.min_window = np.log2(min_window)
        self.max_window = np.log2(max_window) if max_window else int(np.floor(np.log2(N)))

        self.window_sizes = custom_window_list if custom_window_list else 2 ** np.arange(min_window, max_window + 1)
        self.window_scales = np.log2(self.window_sizes)

    def _manage_detrending(self, window):
        """Apply the appropriate detrending method."""
        if self.method == 'SD':
            return window
        elif self.method == 'LD':
            return self._detrend_linear(window)
        elif self.method == 'BD':
            return self._detrend_bridge(window)

    def _detrend_linear(self, window):
        """Remove linear trend from window using regression."""
        x = np.arange(len(window))
        slope, intercept = np.polyfit(x, window, 1)
        trend = slope * x + intercept
        return window - trend

    def _detrend_bridge(self, window):
        """Remove bridge trend from window."""
        if len(window) < 2:
            return window
        x = np.arange(len(window))
        first, last = window[0], window[-1]
        trend = np.linspace(first, last, len(window))
        return window - trend
        
    def _find_exclusions_bounds(self):
        """Determine the exclusions based on the method."""

        lower_window_exclusion = {"SD": [0, 0], "LD": [1, 0], "BD": [1, 0]}

        exclusions_dict = {
            6: {"SD": [0, 2], "LD": [2, 0], "BD": [1, 0]},
            7: {"SD": [0, 3], "LD": [2, 1], "BD": [1, 0]},
            8: {"SD": [0, 3], "LD": [2, 2], "BD": [1, 0]},
            9: {"SD": [1, 4], "LD": [2, 2], "BD": [2, 2]},
            10: {"SD": [1, 4], "LD": [2, 2], "BD": [2, 3]},
            11: {"SD": [1, 5], "LD": [3, 4], "BD": [2, 4]},
            12: {"SD": [1, 5], "LD": [3, 5], "BD": [2, 4]},
            13: {"SD": [2, 6], "LD": [3, 5], "BD": [2, 5]},
            14: {"SD": [2, 7], "LD": [4, 5], "BD": [3, 6]},
            15: {"SD": [2, 7], "LD": [5, 5], "BD": [3, 7]},
            16: {"SD": [3, 7], "LD": [6, 5], "BD": [3, 7]},
            17: {"SD": [4, 7], "LD": [7, 5], "BD": [3, 7]},
        }

        if self.max_window in exclusions_dict:
            return exclusions_dict[self.max_window][self.method]
        elif self.max_window > max(exclusions_dict.keys()):
            return exclusions_dict[max(exclusions_dict.keys())][self.method]
        else:
            return lower_window_exclusion[self.method]


In [32]:
window_sizes =  [2, 4, 5, 8, 10]
np.log2(window_sizes)

array([1.        , 2.        , 2.32192809, 3.        , 3.32192809])

In [ ]:
N = 256
# Choix des tailles de fenêtres : ici, de 2 à N//2 avec un pas de 2.
# (On peut ajuster ce choix en fonction de la série et des recommandations.)
window_sizes = 2 ** np.arange(1, int(np.log2(N)) + 1)
print(f"Window sizes : {window_sizes}")



Window sizes : [np.int64(2), np.int64(4), np.int64(8), np.int64(16), np.int64(32), np.int64(64), np.int64(128), np.int64(256)]


In [16]:
np.log(256, 2)

TypeError: return arrays must be of ArrayType

In [8]:
window_sizes.pop(1)

AttributeError: 'numpy.ndarray' object has no attribute 'pop'